In [3]:
import os
import pandas as pd

In [7]:
def create_df(_data_folder, _result_files):
    # Initialize an empty list to store DataFrames
    result_dfs = []

    # Process each result file
    for file in _result_files:
        # Extract YT_num and rep from the file name
        split_route = file.split("_")
        rep_num = split_route[7].split(".")[0]

        # Read the data from the file into a DataFrame
        file_path = os.path.join(_data_folder, file)
        df = pd.read_csv(file_path)
        df['rep'] = rep_num
        df['file_name'] = file.replace("result-NoCongestions-", "")
        
        # Append the DataFrame to the list
        result_dfs.append(df)

    # Merge all DataFrames
    result_df = pd.concat(result_dfs, ignore_index=True)

    # Display the merged DataFrame
    print("result DataFrame:")
    print(result_df)
    
    return result_df

In [8]:
delay_pos = [
    (250, 0, 0),
    (500, 0, 0),
    (0, 0, 50),
    (250, 0, 50),
    (500, 0, 50),
    (750, 0, 50),
    (0, 0, 100),
    (250, 0, 100),
    (500, 0, 100),
    (750, 0, 100),
    (0, 0, 150),
    (250, 0, 150),
    (500, 0, 150),
    (750, 0, 150),
    (250, 0, 200),
    (500, 0, 200),
    (0, 0, 0),
    (750, 0, 0),
    (750, 0, 200),
    (0, 0, 200)
]

In [12]:
def distance_df(_data_folder, _other_files):
    delay_dfs = []

    for file in _other_files:
        file_path = os.path.join(_data_folder, file)
        _df = pd.read_csv(file_path)
        _df['coordinates'] = _df.apply(lambda row: (row['x'], row['y'], row['z']), axis=1)
    
        # Create 'is_delay_pos' column
        _df['is_delay_pos'] = _df['coordinates'].apply(lambda coord: 1 if coord in delay_pos else 0)

        if 'now' in file:
            _df['Route_id'] = _df['Route_id'] + 100
        
        # series to dataframe
        dist_df = _df['Route_id'].value_counts().rename_axis('Route_id').reset_index(name='dist')
        
        # Count the number of times is_delay_pos is 1 for each Route_id
        delay_count = _df[_df['is_delay_pos'] == 1].groupby('Route_id').size().reset_index(name='delay_count')
        delay_count['Truck_id'] = 'Truck-' + delay_count['Route_id'].astype(str)

        delay_count['file_name'] = file
        # Merge dist_df and delay_count
        delay_count = pd.merge(dist_df, delay_count, on='Route_id')

        delay_dfs.append(delay_count)

    # Merge all DataFrames
    delay_df = pd.concat(delay_dfs, ignore_index=True)
    print("delay DataFrame:")
    print(delay_df)
    
    return delay_df

In [16]:
data_folder = 'original_data'
all_files = os.listdir(data_folder)

In [17]:
# data of simulation results
result_files = [file for file in all_files if "result" in file]

# data of mpnm results
other_files = [file for file in all_files if "result" not in file]

In [18]:
df = create_df(data_folder, result_files)

result DataFrame:
      Truck_id   Route_id             Origin        Destination  Total Time  \
0    Truck-100  Route-100  (425.0 0.0 194.5)   (625.0 0.0 50.0)     361.152   
1    Truck-101  Route-101   (675.0 0.0 55.5)  (625.0 0.0 200.0)     348.368   
2    Truck-102  Route-102  (175.0 0.0 194.5)    (375.0 0.0 0.0)     361.096   
3    Truck-103  Route-103   (675.0 0.0 -5.5)  (125.0 0.0 200.0)     380.200   
4    Truck-104  Route-104   (425.0 0.0 55.5)   (375.0 0.0 50.0)     378.624   
..         ...        ...                ...                ...         ...   
715   Truck-15   Route-15  (675.0 0.0 205.5)    (625.0 0.0 0.0)     349.912   
716   Truck-16   Route-16   (675.0 0.0 55.5)  (375.0 0.0 200.0)     347.896   
717   Truck-17   Route-17  (175.0 0.0 155.5)   (125.0 0.0 50.0)     371.592   
718   Truck-18   Route-18  (425.0 0.0 105.5)    (375.0 0.0 0.0)     381.832   
719   Truck-19   Route-19  (175.0 0.0 205.5)  (625.0 0.0 100.0)     398.296   

     PickupSta AT  DropSta AT   r

In [19]:
delay_df = distance_df(data_folder, other_files)

delay DataFrame:
     Route_id  dist  delay_count   Truck_id  \
0         115    53            9  Truck-115   
1         103    37            7  Truck-103   
2         111    35           10  Truck-111   
3         104    35            8  Truck-104   
4         110    33            7  Truck-110   
..        ...   ...          ...        ...   
715        16    19            4   Truck-16   
716         5    17            3    Truck-5   
717        13    17            3   Truck-13   
718         7    15            2    Truck-7   
719        11    15            2   Truck-11   

                                    file_name  
0      now_RoutePoints_20_LP_0_0_100_1rep.csv  
1      now_RoutePoints_20_LP_0_0_100_1rep.csv  
2      now_RoutePoints_20_LP_0_0_100_1rep.csv  
3      now_RoutePoints_20_LP_0_0_100_1rep.csv  
4      now_RoutePoints_20_LP_0_0_100_1rep.csv  
..                                        ...  
715  prev_RoutePoints_20_LP_80_10_10_2rep.csv  
716  prev_RoutePoints_20_LP_80_10_

In [20]:
# Merge on 'file_name' and 'Truck_id'
merged_df = pd.merge(df, delay_df, on=['Truck_id','file_name'])
# merged_df = merged_df[['file_name', 'Truck_id', 'dist', 'delay_count', 'Total Time']]
merged_df = merged_df[['dist', 'delay_count', 'Total Time']]

# insert column at specific index location
merged_df.insert(0, 'Truck_id', 'Truck-' + (merged_df.index).astype(str))
print("merged DataFrame:")
print(merged_df)

merged DataFrame:
      Truck_id  dist  delay_count  Total Time
0      Truck-0    25            5     361.152
1      Truck-1    19            4     348.368
2      Truck-2    27            6     361.096
3      Truck-3    37            7     380.200
4      Truck-4    35            8     378.624
..         ...   ...          ...         ...
715  Truck-715    21            5     349.912
716  Truck-716    19            4     347.896
717  Truck-717    31            6     371.592
718  Truck-718    35            8     381.832
719  Truck-719    45            9     398.296

[720 rows x 4 columns]


In [45]:
# to csv file
# merged_df.to_csv('data.csv', index=False)

In [56]:
# columns = ['Truck_id', 'Completion_Time']

In [57]:
# folder_path = 'Results_old_2'

# all_data = pd.DataFrame()

# # read subfolder of folder
# all_folders = os.listdir(folder_path)

# # read all files in subfolder
# all_files = []
# for folder in all_folders:
#     _folder_path = os.path.join(folder_path, folder)
#     # read only csv file
#     csv_files = [file for file in os.listdir(_folder_path) if file.endswith('.csv')]
    
#     # read csv file and merge them
#     for file in csv_files:
#         file_path = os.path.join(_folder_path, file)
#         # read csv file to dataframe and select columns
#         _df = pd.read_csv(file_path)
#         _df = _df[columns]
#         _df['file_name'] = file
#         _df['file_name'].replace('result-', '')
#         _df['Truck_id_number'] = df['Truck_id'].apply(lambda x: int(x.split('-')[1]))
        
#         _df.loc[df['Truck_id_number'] >= 100]['file_name'] = _df['file_name'].replace()
#         # replace file name to blank
#         # _df['file_name'] = file.replace('result_', '')
        
#         # # check the number next to 'Truck-'
#         # if int(_df['Truck_id'].str.extract(r'(\d+)')) <= 100:
#         #     _df['file_name']
        
#         # print(_df.head())
        
#         all_data = pd.concat([all_data, _df])
        
# all_data.head()


In [77]:
# data of mpnm results
import glob

folder_path = '../Simulation/Assets/Data/old_data/Congestion_dh'

all_files_df = pd.DataFrame()

# read subfolder of folder not meta
all_folders = [folder for folder in os.listdir(folder_path) if not folder.endswith('.meta')]

all_files_list = []

for folder in all_folders:
    _folder_path = os.path.join(folder_path, folder)
    
    for subfolder in os.listdir(_folder_path):
        # read all folder in subfolder not meta (now, prev)
        _subfolders = [subfolder for subfolder in os.listdir(_folder_path) if not subfolder.endswith('.meta')]
        
        # read csv file in subfolder
        for sub in _subfolders:
            _subfolder_path = os.path.join(_folder_path, sub)
            csv_file_path_list = glob.glob(os.path.join(_subfolder_path, "*RoutePoints*.csv"))
            
            # concat the list
            all_files_list.extend(csv_file_path_list)

print(all_files_list[:5])

['../Simulation/Assets/Data/old_data/Congestion_dh\\prev_10_now_10\\now_10\\now_RoutePoints_10_LP_0_0_100_43rep.csv', '../Simulation/Assets/Data/old_data/Congestion_dh\\prev_10_now_10\\now_10\\now_RoutePoints_10_LP_0_0_100_44rep.csv', '../Simulation/Assets/Data/old_data/Congestion_dh\\prev_10_now_10\\now_10\\now_RoutePoints_10_LP_0_0_100_45rep.csv', '../Simulation/Assets/Data/old_data/Congestion_dh\\prev_10_now_10\\now_10\\now_RoutePoints_10_LP_0_0_100_46rep.csv', '../Simulation/Assets/Data/old_data/Congestion_dh\\prev_10_now_10\\now_10\\now_RoutePoints_10_LP_0_0_100_47rep.csv']


In [73]:
def distance_df_2(_files_list):
    delay_dfs = []

    for file_path in _files_list:
        _df = pd.read_csv(file_path)
        _file_name = os.path.basename(file_path)
        _df['coordinates'] = _df.apply(lambda row: (row['x'], row['y'], row['z']), axis=1)
    
        # Create 'is_delay_pos' column
        _df['is_delay_pos'] = _df['coordinates'].apply(lambda coord: 1 if coord in delay_pos else 0)

        if 'now' in _file_name:
            _df['Route_id'] = _df['Route_id'] + 100
        
        # series to dataframe
        dist_df = _df['Route_id'].value_counts().rename_axis('Route_id').reset_index(name='dist')
        
        # Count the number of times is_delay_pos is 1 for each Route_id
        delay_count = _df[_df['is_delay_pos'] == 1].groupby('Route_id').size().reset_index(name='delay_count')
        delay_count['Truck_id'] = 'Truck-' + delay_count['Route_id'].astype(str)

        # get file name from file path
        delay_count['file_name'] = _file_name
        # Merge dist_df and delay_count
        delay_count = pd.merge(dist_df, delay_count, on='Route_id')

        delay_dfs.append(delay_count)

    # Merge all DataFrames
    delay_df = pd.concat(delay_dfs, ignore_index=True)
    print("delay DataFrame:")
    print(delay_df)
    
    return delay_df

In [80]:
df = distance_df_2(all_files_list)

delay DataFrame:
        Route_id  dist  delay_count   Truck_id  \
0            100    53           11  Truck-100   
1            104    41            7  Truck-104   
2            105    37            5  Truck-105   
3            107    37            7  Truck-107   
4            109    37            7  Truck-109   
...          ...   ...          ...        ...   
249475         1    53            9    Truck-1   
249476         2    43            8    Truck-2   
249477         3    43           10    Truck-3   
249478         0    27            6    Truck-0   
249479         4    25            3    Truck-4   

                                       file_name  
0        now_RoutePoints_10_LP_0_0_100_43rep.csv  
1        now_RoutePoints_10_LP_0_0_100_43rep.csv  
2        now_RoutePoints_10_LP_0_0_100_43rep.csv  
3        now_RoutePoints_10_LP_0_0_100_43rep.csv  
4        now_RoutePoints_10_LP_0_0_100_43rep.csv  
...                                          ...  
249475  prev_RoutePoints_

In [81]:
# to csv file
df.to_csv('results_old_2.csv', index=False)